This initial concept is based upon [this Medium post](https://medium.com/@curiousily/making-a-predictive-keyboard-using-recurrent-neural-networks-tensorflow-for-hackers-part-v-3f238d824218) by Venelin Valkov. The model from the article has been modified in this notebook to allow multiple LSTM layers, and to accept prediction inputs of less than the maximum sequence length.

 - Use a pre-trained model so that training starts from a better position

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import nltk

# Import Corpuses

Next we will import the corpuses that we are going to use to train our model. These corpuses are taken from Project Gutenberg.

In [3]:
data = pd.read_csv("../raw_data/wiki_movie_plots_deduped.csv")

In [4]:
data.Plot.head()

0    A bartender is working at a saloon, serving dr...
1    The moon, painted with a smiling face hangs ov...
2    The film, just over a minute long, is composed...
3    Lasting just 61 seconds and consisting of two ...
4    The earliest known adaptation of the classic f...
Name: Plot, dtype: object

In [5]:
#nltk.download('gutenberg')
corpuses = [plot for plot in data.Plot]
#corpus = list(nltk.corpus.gutenberg.words('carroll-alice.txt'))
#corpus_string = ' '.join(corpus)

print(len(corpuses), 'corpuses found')
print(sum([len(x) for x in corpuses]), 'characters in corpuses')

34886 corpuses found
75529395 characters in corpuses


In [6]:
#reduce dataset to work
corpuses = corpuses[0:100]

In [7]:
corpuses

["A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]",
 "The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.",
 'The film, just over a minute long, is composed of t

We will need a unique list of characters that are found in our corpus. We will also need two dictionaries that allow us to convert our characters into unique integers and vice versa, so they will also be created below. We define two extra characters that can be used by our model; the first is a padding character so that we can take smaller inputs, while the second is an unknown character incase we come across characters that were not in our training data.

In [8]:
PADDING_CHAR = -1
UNKNOWN_CHAR = -2

In [9]:
characters = sorted(list(set(''.join(corpuses))))
char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

char_indices[''] = PADDING_CHAR
indices_char[PADDING_CHAR] = ''
indices_char[UNKNOWN_CHAR] = ''

print(len(characters), 'unique characters in the corpus')

90 unique characters in the corpus


In [10]:
def string_to_indices(string, ind_map=char_indices):
    return [ind_map[c] if c in ind_map else UNKNOWN_CHAR for c in string]

def indices_to_string(indices, char_map=indices_char):
    return ''.join([char_map[i] for i in indices])

def char_to_probabilities(char, chars=characters):
    return [1 if c==char else 0 for c in chars]

# Prepare Training Data

Now we will split our corpus into training data, where each data point has a size SEQUENCE_LENGTH. In this case we will use a sequence length of 40, meaning our predictions will be based on 40 previous characters. STEP allows us to choose how many data points we have, as each data point will be separated by STEP characters. Setting STEP to 1 allows us to select every possible point.

In [11]:
SEQUENCE_LENGTH = 40
STEP = 1

sentences = []
next_chars = []
for corpus_string in corpuses:
    for i in range(0, len(corpus_string) - 1, STEP):
        sentences.append(corpus_string[i: i + SEQUENCE_LENGTH])
        next_chars.append(corpus_string[min(i + SEQUENCE_LENGTH, len(corpus_string) - 1)])

print(len(sentences), 'training data points')

87171 training data points


The following method allows us to convert our text into a list of indices that can be understood by our model. We will then prepare our sentences and characters from our corpus so they can be used to train our model.

In [12]:
def prepare_text_indices(text):
    ind_text = string_to_indices(text)
    if len(ind_text) < SEQUENCE_LENGTH:
        ind_text = ([PADDING_CHAR] * (SEQUENCE_LENGTH - len(ind_text))) + ind_text
    elif len(ind_text) > SEQUENCE_LENGTH:
        ind_text = ind_text[-SEQUENCE_LENGTH:]
    return np.array(ind_text) / len(characters)

In [13]:
X_train = np.array([prepare_text_indices(s) for s in sentences])
X_train = X_train.reshape((X_train.shape[0], SEQUENCE_LENGTH, 1))
Y_train = np.array([char_to_probabilities(c) for c in next_chars])

# Build Model

The following cells will be used to build and train our model. The first cell defines our model architecture. It also allows us to load model weights if we have saved them previously. This method is used in the second cell, where we train our model and then reload the best weights (in terms of training loss).

In [14]:
#pip install tensorflow

In [15]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

def build_model(input_shape, output_shape, weight_path=None):
    model = Sequential()
    model.add(LSTM(512, input_shape=input_shape, return_sequences=True, dropout=0.25))
    #model.add(LSTM(512, return_sequences=True, dropout=0.25))
    model.add(LSTM(512, dropout=0.25))
    model.add(Dense(output_shape))
    model.add(Activation('softmax'))
    
    if weight_path != None:
        model.load_weights(weight_path)
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

2022-09-06 12:25:33.834777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-06 12:25:33.834842: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [16]:
# Get model architecture
model = build_model((SEQUENCE_LENGTH, 1), len(characters)) #weight_path='../input/predictive-text-model-weights/weights-512-512.hdf5'

# Fit model and save best weights
#checkpoint = ModelCheckpoint('weights-512-512.hdf5', monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(X_train, Y_train, batch_size=512, epochs=10, shuffle=True) #epochs reduced to 10      #, callbacks=[checkpoint]

# Load trained weights
model = build_model((SEQUENCE_LENGTH, 1), len(characters)) #weight_path='weights-512-512.hdf5'

2022-09-06 12:25:40.694948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-06 12:25:40.695099: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-06 12:25:40.695150: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-LCPJ8NA): /proc/driver/nvidia/version does not exist
2022-09-06 12:25:40.697307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
171/171 [==============================] - 1966s 11s/step - loss: 3.1500 - accuracy: 0.1646
Epoch 2/10
171/171 [==============================] - 1355s 8s/step - loss: 3.1036 - accuracy: 0.1665
Epoch 3/10
171/171 [==============================] - 1083s 6s/step - loss: 3.0906 - accuracy: 0.1662
Epoch 4/10
171/171 [==============================] - 2089s 12s/step - loss: 3.0599 - accuracy: 0.1665
Epoch 5/10
171/171 [==============================] - 2236s 13s/step - loss: 2.9701 - accuracy: 0.1832
Epoch 6/10
171/171 [==============================] - 1121s 7s/step - loss: 2.8982 - accuracy: 0.1973
Epoch 7/10
171/171 [==============================] - 1903s 11s/step - loss: 2.8780 - accuracy: 0.2004
Epoch 8/10
171/171 [==============================] - 1309s 8s/step - loss: 2.8639 - accuracy: 0.2013
Epoch 9/10
171/171 [==============================] - 1662s 10s/step - loss: 2.8515 - accuracy: 0.2041
Epoch 10/10
171/171 [==============================] - 1554s 9s/step - loss: 

# Make Predictions
We are going to create two functions that will allow us to make predictions. The first function will take our text and predict the next character. The second function will take our text and predict the next word (or word ending) by making use of our first function.

In [17]:
def make_char_prediction(text, model=model, pred_length=SEQUENCE_LENGTH):
    ind_text = prepare_text_indices(text)
        
    X_text = np.array(ind_text).reshape((1, pred_length, 1))
    pred = model.predict(X_text)
    top_pred = np.argmax(pred)
    char_pred = characters[top_pred]
    return char_pred

def make_word_prediction(text, model=model, min_length=2, pred_length=SEQUENCE_LENGTH):
    pred_text = ''
    while True:
        x = (text + pred_text)[-SEQUENCE_LENGTH:]
        pred = make_char_prediction(x, model)
        pred_text += pred
        if len(pred_text) >= min_length and (pred == ' '):
            return pred_text

Now we will take a few examples and make some predictions.

In [ ]:
test_texts = ["I\'ve believed as many as six impossible ",
              "no use going back to yesterday, because ",
              "A pessimist sees the difficulty in every",
              "sometimes you just have to create a rand",
              "The score was nil nil until the striker ",
              "if I could dream of such things then the",
              "We should follow the path on the left as",
              "Let\'s write a short program to display a",
              "no"]

test_preds = []
test_preds_long = []
for text in test_texts:
    test_preds.append(make_word_prediction(text, model=model))
    test_preds_long.append(make_word_prediction(text, model=model, min_length=10))

pd.DataFrame({'Text' : test_texts, 'Prediction' : test_preds, 'Long Prediction' : test_preds_long})

1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 76ms/step


1/1 [==============================] - 0s 193ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 102ms/step


...

1/1 [==============================] - 0s 79ms/step


1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 73ms/step


In [ ]:
long_test_text = 'We should follow the path on the left as'
new_long_test_text = long_test_text + make_word_prediction(long_test_text, model=model, min_length=20)

print('Original:', long_test_text)
print('New:     ', new_long_test_text)